Note: This notebook requires Python Verison == 3.9

In [1]:
#download deep walk repo and unzip
!wget https://github.com/phanein/deepwalk/archive/refs/heads/master.zip 
!unzip master.zip

--2024-04-18 09:51:21--  https://github.com/phanein/deepwalk/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/phanein/deepwalk/zip/refs/heads/master [following]
--2024-04-18 09:51:21--  https://codeload.github.com/phanein/deepwalk/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ]   1.27M  7.31MB/s    in 0.2s    

2024-04-18 09:51:21 (7.31 MB/s) - ‘master.zip’ saved [1330818]

Archive:  master.zip
6e6dff245e4692e9bea47e9017c1034e51afbf29
   creating: deepwalk-master/
  inflating: deepwalk-master/.gitignore  
  inflating

In [2]:
#download paper's repo and unzip
!wget https://github.com/zhchs/Disease-Prediction-via-GCN/archive/refs/heads/main.zip
!unzip main.zip

--2024-04-18 09:51:22--  https://github.com/zhchs/Disease-Prediction-via-GCN/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/zhchs/Disease-Prediction-via-GCN/zip/refs/heads/main [following]
--2024-04-18 09:51:22--  https://codeload.github.com/zhchs/Disease-Prediction-via-GCN/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]  36.79K  --.-KB/s    in 0.03s   

2024-04-18 09:51:22 (1.28 MB/s) - ‘main.zip’ saved [37668]

Archive:  main.zip
779c5c17d12aa2b5263ae2f2a2883d4f17cd5d31
   creating: Disease-Prediction-via-GCN-main/
   

In [3]:
import os
import shutil

#copy data to working folder and move there
shutil.copytree('./Disease-Prediction-via-GCN-main/data', './deepwalk-master/data')
os.chdir("./deepwalk-master/")

In [4]:
#install requirements
!pip install -r requirements.txt

In [5]:
#install deepwalk
!python setup.py install

running install
/home/dlochner3/~miniconda3/envs/deepwalk/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/home/dlochner3/~miniconda3/envs/deepwalk/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installe

In [6]:
#down grade packages as neccesary
!pip uninstall scipy -y
!pip install scipy==1.10.1

!pip uninstall gensim -y
!pip install gensim==3.8.3

!pip install scikit-learn

!pip install pandas

Found existing installation: scipy 1.10.1
Uninstalling scipy-1.10.1:
  Successfully uninstalled scipy-1.10.1
  Using cached scipy-1.10.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached scipy-1.10.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
Found existing installation: gensim 3.8.3
Uninstalling gensim-3.8.3:
  Successfully uninstalled gensim-3.8.3
  Using cached gensim-3.8.3-cp39-cp39-linux_x86_64.whl


In [7]:
#read in data from paper

import pickle

path = './data/sample_data/'

# map of nodes to index
with open(path + "sample_garph.map.pkl", "rb") as f:
     map = pickle.load(f)

# adj list of nodes:
# {node1(str): [neighbor1(str), neighbor2(str), ...], node2: []...}
with open(path + "sample_garph.adj.pkl", "rb") as f:
     adj = pickle.load(f)

# rare flag, indicating whether a node is a rare disease (value=1)
# or contains a rare disease, NumPy array of shape (N * 1)
with open(path + "sample_garph.rare.label.pkl", "rb") as f:
     rarelabel = pickle.load(f)

# list of node: [node1(str), node2(str), node3(str), ...]
with open(path + "sample_garph.nodes.pkl", "rb") as f:
     nodes = pickle.load(f)

# NumPy array of shape (N * D), N is node number
# and D is the number of diseases
with open(path + "sample_garph.label.pkl", "rb") as f:
     label = pickle.load(f)

# list of nodes for training, [node_idx_1(int), node_idx_2(int), ....]
with open(path + "sample_garph.test.pkl", "rb") as f:
     test_idx = pickle.load(f)

# list of nodes for testing, [node_idx_1(int), node_idx_2(int), ....]
with open(path + "sample_garph.train.pkl", "rb") as f:
     train_idx = pickle.load(f)

In [8]:
import pandas as pd
import numpy as np


#get adjcency matrix into proper format for deepwalk
adj_long = []
for patient in adj:
    for i in adj[patient]:
        adj_long.append([patient, i])
        
adj_df = pd.DataFrame(adj_long, columns=['from','to']).sort_values(['from', 'to'])
adj_df = adj_df[adj_df['from']<= 805]
adj_df.to_csv('fulldata.txt', index=False, header=False, sep='\t')

In [9]:
#run deep walk
!deepwalk --format adjlist --input fulldata.txt --max-memory-data-size 0 --number-walks 80 --representation-size 128 --walk-length 40 --window-size 10 --workers 7 --output disease.embedings

Number of nodes: 926
Number of walks: 74080
Data size (walks*length): 2963200
Data size 2963200 is larger than limit (max-memory-data-size: 0).  Dumping walks to disk.
Walking...
Counting vertex frequency...
Training...


In [10]:
from gensim.models import KeyedVectors

#load deep walk model
embeddings_file = 'disease.embedings'
model = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)

In [11]:
#get deepwalk embeddings
feats = np.asarray([model[str(node)] for node in range(1+max(adj_df['from']))])
feats.shape

(806, 128)

In [12]:
from scipy import sparse

#create train, test, and rare test split arrays
X_train, X_test = feats[[i for i in train_idx if i <= 805]], feats[[i for i in test_idx if i <= 805]]
y_train, y_test = label[[i for i in train_idx if i <= 805]], label[[i for i in test_idx if i <= 805]]

X_test_rare = feats[[i for i, j in zip(test_idx, rarelabel) if i <=805 and j==1]]
y_test_rare = label[[i for i, j in zip(test_idx, rarelabel) if i <=805 and j==1]]

#set train y to sparse
y_train = sparse.csr_matrix(y_train)

In [14]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

#create classifier, train, and predict on all data
clf = OneVsRestClassifier(LogisticRegression())
clf.fit(X_train, y_train)
preds = clf.predict_proba(X_test)

/home/dlochner3/~miniconda3/envs/deepwalk/lib/python3.9/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 22 is present in all training examples.
  warnings.warn(
/home/dlochner3/~miniconda3/envs/deepwalk/lib/python3.9/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 34 is present in all training examples.
  warnings.warn(
/home/dlochner3/~miniconda3/envs/deepwalk/lib/python3.9/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 37 is present in all training examples.
  warnings.warn(
/home/dlochner3/~miniconda3/envs/deepwalk/lib/python3.9/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 43 is present in all training examples.
  warnings.warn(
/home/dlochner3/~miniconda3/envs/deepwalk/lib/python3.9/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 44 is present in all training examples.
  warnings.warn(
/home/dlochner3/~miniconda3/envs/deepwalk/lib/python3.9/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 4

In [15]:
#create evaluation metric
def metrics(preds, actual, topk = [1,2,3,4,5]):
  results = []
  maxk = max(topk)
  pred_k = np.argsort(-1*preds)[:,0:maxk]
  for k in topk:
    tp = 0
    for patient in range(preds.shape[0]):
      for idx in range(k):
        disease = pred_k[patient, idx]
        if actual[patient, disease] == 1:
          tp += 1
    precision = tp / (k * preds.shape[0])
    recall = tp / actual.sum()
    f1 = 2*precision*recall/(precision + recall)
    results.append([k, f1, precision, recall])
  results = pd.DataFrame(results, columns=['k', 'f1', 'prec','rec']).round(4)
  return results

In [16]:
#get evaluation on full data
print(metrics(preds, y_test))

   k      f1    prec     rec
0  1  0.3107  0.3967  0.2553
1  2  0.3558  0.3161  0.4069
2  3  0.3557  0.2700  0.5213
3  4  0.3185  0.2211  0.5691
4  5  0.2951  0.1934  0.6223


In [17]:
#get evaluation for rare patients only
preds_rare = clf.predict_proba(X_test_rare)
print(metrics(preds_rare, y_test_rare))

   k      f1    prec     rec
0  1  0.3148  0.3902  0.2637
1  2  0.3645  0.3171  0.4286
2  3  0.3666  0.2737  0.5549
3  4  0.3294  0.2256  0.6099
4  5  0.3137  0.2033  0.6868
